# Fragment Decomposition

Abhinav Madahar <abhinav.madahar@rutgers.edu>, James Abello Monedero <abelloj@cs.rutgers.edu>

<br />

We want to find the fragment decomposition of a large graph.

In [3]:
import networkx as nx
import matplotlib.pyplot as plt
from queue import Queue
from unionfind import UnionFind

First, we need to be able to run pBFS on a path in a graph and get the connected components of the waves.
Let's make an implementation of pBFS which yields tuples of connected components at every iteration.

In [130]:
from collections import namedtuple


Node = namedtuple('Node', ['uid', 'children'])

def parallel_bfs(G: nx.Graph, path: list):
    queue = Queue()
    for source in path:
        queue.put(source)
    
    roots = (Node(source, []) for source in path)
    tree = {source: root for source, root in zip(path, roots)}
    
    while not queue.empty():
        v = queue.get()
        for w in G.adj[v]:
            if w not in tree:
                tree[w] = Node(w, [])
                tree[v].children.append(w)
                queue.put(w)
    
    def get_underlying_subtree(head: Node) -> Node:
        return Node(head.uid, [get_underlying_subtree(tree[child]) for child in head.children])
    
    pbfs = [get_underlying_subtree(tree[source]) for source in path]
    
    waves = {}

    def add_to_tree(head, parent: int = None, depth: int = 0):
        if depth not in waves:
            waves[depth] = []
        waves[depth].append((head.uid, parent))
        for child in head.children:
            add_to_tree(child, head.uid, depth + 1)

    for pbfs_section in pbfs:
        add_to_tree(pbfs_section)

    return waves

Let's get the waves.

In [139]:
G = nx.gnm_random_graph(100, 200)
G = G.subgraph(next(nx.connected_components(G))).copy()
G.add_edge(0, 1)
G.add_edge(1, 2)
G.add_edge(2, 3)
G.add_edge(3, 4)
pbfs = parallel_bfs(G, [0, 1, 2, 3, 4])
tree = nx.DiGraph()
! rm -f pbfs.dot
with open('pbfs.dot', 'a') as pbfs_file:
    pbfs_file.write('digraph G {')
    for layer in pbfs.values():
        for dest, src in layer:
            if src:
                pbfs_file.write(f'{src} -> {dest};')
    pbfs_file.write('}')
            
import subprocess
subprocess.call(["dot", "-Tpng", "pbfs.dot", "-o", "pbfs.png"]) 

0